# Snaike game with neural nets

## Importing modules

In [1]:
from Snake import Snake
from body import body
from head import head
from food import food
from SnakeTools import *
from SnakeRender import *
from pygame.locals import *
from SnakeAi import *
from PopulationTools import *
import pygame,sys
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import gc
import os
import threading

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
#colors
WHITE = (255, 255, 255)
BLACK=(0,0,0)
RED= (220,20,60)

In [3]:
#Session
session = tf.Session()
session.run(tf.global_variables_initializer())

In [4]:
#Neural net configuration
inputlayer=4
layer1=10
layer2=3
chromsize=83

In [5]:
#Placeholder
x=tf.placeholder(tf.float32,shape=[None,inputlayer], name='x')

def CreateNeuralNet(chromossome):
    #The first (inputlayer*layer1) numbers are the weights of the first layer, and the next (layer1) numbers are their biases
    #This logic is applyied to the next layers
    w1=chromossome[0:(inputlayer*layer1)].reshape([inputlayer,layer1])
    b1=chromossome[(inputlayer*layer1):(inputlayer*layer1+layer1)]
    w2=chromossome[(inputlayer*layer1+layer1):(inputlayer*layer1+layer1+layer1*layer2)].reshape([layer1,layer2])
    b2=chromossome[(inputlayer*layer1+layer1+layer1*layer2):(inputlayer*layer1+layer1+layer1*layer2+layer2)]
    
    #Neural net definition
    w1=tf.Variable(w1,dtype=tf.float32)
    b1=tf.Variable(b1,dtype=tf.float32)
    first_layer=tf.nn.relu(tf.matmul(x,w1)+b1)
    w2=tf.Variable(w2,dtype=tf.float32)
    b2=tf.Variable(b2,dtype=tf.float32)
    second_layer=tf.nn.softmax(tf.matmul(first_layer,w2)+b2)
    out_layer=tf.argmax(second_layer,axis=1)
    session.run(tf.global_variables_initializer())
    
    #The tensor that is returned outputs the class directly
    return out_layer

In [6]:
#init
print("Initializing game")
pygame.init()


#Font config
pygame.font.init() 
myfont = pygame.font.SysFont('Comic Sans MS', 25)
game_name = myfont.render('Snaike Game', False, (220,20,60))
myfont2 = pygame.font.SysFont('Comic Sans MS', 25)

#Screen config
dim=500
dim2=int(dim/2)
displaysurf=pygame.display.set_mode((dim,dim+150))
pygame.display.set_caption('Snaike')

#FPS
fpsClock = pygame.time.Clock()
FPS = 45


#Pool
pool=SubjectPool()

Initializing game
['Gen_296_.gen', 'Gen_295_.gen', 'Gen_294_.gen', 'Gen_293_.gen', 'Gen_292_.gen', 'Gen_291_.gen', 'Gen_290_.gen', 'Gen_289_.gen', 'Gen_288_.gen', 'Gen_287_.gen', 'Gen_286_.gen', 'Gen_285_.gen', 'Gen_284_.gen', 'Gen_283_.gen', 'Gen_282_.gen', 'Gen_281_.gen', 'Gen_280_.gen', 'Gen_279_.gen', 'Gen_278_.gen', 'Gen_277_.gen', 'Gen_276_.gen', 'Gen_275_.gen', 'Gen_274_.gen', 'Gen_273_.gen', 'Gen_272_.gen', 'Gen_271_.gen', 'Gen_270_.gen', 'Gen_269_.gen', 'Gen_268_.gen', 'Gen_267_.gen', 'Gen_266_.gen', 'Gen_265_.gen', 'Gen_264_.gen', 'Gen_263_.gen', 'Gen_262_.gen', 'Gen_261_.gen', 'Gen_260_.gen', 'Gen_259_.gen', 'Gen_258_.gen', 'Gen_257_.gen', 'Gen_256_.gen', 'Gen_255_.gen', 'Gen_254_.gen', 'Gen_253_.gen', 'Gen_252_.gen', 'Gen_251_.gen', 'Gen_250_.gen', 'Gen_249_.gen', 'Gen_248_.gen', 'Gen_247_.gen', 'Gen_246_.gen', 'Gen_245_.gen', 'Gen_244_.gen', 'Gen_243_.gen', 'Gen_242_.gen', 'Gen_241_.gen', 'Gen_240_.gen', 'Gen_239_.gen', 'Gen_238_.gen', 'Gen_237_.gen', 'Gen_236_.gen', 'Gen_

In [7]:
class SnakeThread(threading.Thread):
    
    def __init__(self,id):
        threading.Thread.__init__(self)
        self.id=id
    def run(self):
        
        #Score
        score=0
        attempt=1
        attempt_flag=True
        framecount=0
        
        #emilia variables
        emiliaImg = pygame.image.load('blue.png')
        emiliaSnake = Snake(int(dim/2),int(dim/2),emiliaImg,ori='r')
        emiliaSnake.createWalls(dim)

        #subaro
        subaroImg=pygame.image.load('blue.png')
        emiliaSnake.bodyImg=subaroImg

        #game over image
        gameoverImg=pygame.image.load('gameover.jpg')

        #food image
        foodImg=pygame.image.load('hamb.png')
        foood=newFood(foodImg,dim)

        #orientation
        lastorientation=emiliaSnake.orientation
        orientation=emiliaSnake.orientation
        gameOverFlag=False


        #subject
        subject=pool.get_subj()
        neural_net=CreateNeuralNet(subject[1])

        #Defined as time + score*time
        fitness=0
        timeout=1.5
        food_time=time.time()
        hamb=0
        emiliaSnake.addBlock()
        garbage_time=time.time()

        #game loop
        print("game starts")
        Snake_time=time.time()

        while True:   

            #Ordinary display
            displaysurf.fill(WHITE)
            pygame.draw.line(displaysurf,BLACK,(0,dim),(dim,dim))
            displaysurf.blit(game_name,(10,dim+10))
            game_score= myfont2.render('Score: '+str(score), False, (0, 0, 0))
            displaysurf.blit(game_score,(10,dim+40))
            now=time.time()
            game_time= myfont2.render('Time: '+str(int(now-Snake_time)), False, (0, 0, 0))
            displaysurf.blit(game_time,(10,dim+70))
            game_attempt= myfont2.render('Attempt: '+str(attempt), False, (0, 0, 0))
            displaysurf.blit(game_attempt,(10,dim+100))
            game_subject= myfont2.render('Subject: '+subject[0], False, (0, 0, 0))   
            displaysurf.blit(game_subject,(dim2-50,dim+10))
            timedelta=now-Snake_time
            game_fitness=myfont2.render('Fitness: '+str(round(fitness,3)), False, (0, 0, 0))   
            displaysurf.blit(game_fitness,(dim2-50,dim+40))
            game_hamb=myfont2.render('Hamb eaten: '+str(hamb), False, (0, 0, 0))   
            displaysurf.blit(game_hamb,(dim2-50,dim+70))

            for event in pygame.event.get():
                if event.type== QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type==KEYDOWN:
                    if event.key==K_UP and orientation!='d':
                        orientation='u'
                    elif event.key==K_DOWN and orientation!='u':
                        orientation='d'
                    elif event.key==K_RIGHT and orientation!='l':
                        orientation='r'
                    elif event.key==K_LEFT and orientation!='r':
                        orientation='l'
                    elif event.key==K_f:
                        emiliaSnake = Snake(int(dim/2),int(dim/2),emiliaImg)
                        emiliaSnake.createWalls(dim)
                        emiliaSnake.bodyImg=subaroImg
                        emiliaSnake.addBlock()
                        gameOverFlag=False
                        attempt_flag=True
                        food_time=time.time()
                        subject=pool.get_subj()
                        neural_net=CreateNeuralNet(subject[1])

                    elif event.key==K_n:
                        pygame.quit()
                        sys.exit()

            framecount=(framecount+1)%2
            fitness=timedelta*(score+1)
            fooddelta=now-food_time
            if(fooddelta>=timeout):
                emiliaSnake.alive=False

            #emilia motion
            if(emiliaSnake.isAlive()):


                if(framecount==0):
                    atr1,atr2,atr3=emiliaSnake.getAtr()
                    ang,_= foodPos(emiliaSnake,foood)
                    ang=int(ang)
                    features=[[atr1,atr2,atr3,ang]] 

                    mv=session.run(neural_net,feed_dict={x:features})-1
                    if(orientation=='u' and mv==1):
                        orientation='r'
                    elif(orientation=='r' and mv==-1):
                        orientation='u'
                    else:
                        orientation=numbOri(mv+oriNumb(orientation))

                emiliaSnake.move(orientation)
                emiliaSnake.move(orientation)

            if(not emiliaSnake.isAlive()):
                gameOverFlag=True
                pool.set_fitness(fitness,self.id)
                break

            if(emiliaSnake.eaten(foood)):

                foood=newFood(foodImg,dim)
                score+=1
                hamb+=1
                food_time=time.time()
                emiliaSnake.addBlock()

            if(not gameOverFlag):

                renderSnake(displaysurf,emiliaSnake)
                renderFood(displaysurf,foood)
                pygame.display.update()






In [8]:
for _ in range(10):
    print("One generation")
    threads=[]
    for i in range(100):
        print("Thread",i)
        thread=SnakeThread(i)
        threads.append(thread)
        thread.run()

    for t in threads:
        t.join()

One generation
Thread 0
game starts
Thread 1
game starts
Thread 2
game starts
Thread 3
game starts
Thread 4
game starts
Thread 5
game starts
Thread 6
game starts
Thread 7
game starts
Thread 8
game starts
Thread 9
game starts
Thread 10
game starts
Thread 11
game starts
Thread 12
game starts
Thread 13
game starts
Thread 14
game starts
Thread 15
game starts
Thread 16
game starts
Thread 17
game starts
Thread 18
game starts
Thread 19
game starts
Thread 20
game starts
Thread 21
game starts
Thread 22
game starts
Thread 23
game starts
Thread 24
game starts
Thread 25
game starts
Thread 26
game starts
Thread 27
game starts
Thread 28
game starts
Thread 29
game starts
Thread 30
game starts
Thread 31
game starts
Thread 32
game starts
Thread 33
game starts
Thread 34
game starts
Thread 35
game starts
Thread 36
game starts
Thread 37
game starts
Thread 38
game starts
Thread 39
game starts
Thread 40
game starts
Thread 41
game starts
Thread 42
game starts
Thread 43
game starts
Thread 44
game starts
Threa

RuntimeError: cannot join thread before it is started